# Getting Data

run  docker run -p 6379:6379 -it --rm redislabs/redistimeseries

In [1]:
import requests
import json
import user
import time
from datetime import datetime

In [26]:
# Getting input for the company symbol
company_symbol="IBM"

In [33]:
# Requesting
req_url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+company_symbol+'&outputsize=full&&apikey='+user.api
resp = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+company_symbol+'&outputsize=full&&apikey='+user.api)

# Check status code
if resp.status_code != 200:
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
#     What does the above code do?
data = resp.json()

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&outputsize=full&&apikey=XYKFTUH8J9GE0HX9


In [34]:
data

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2021-01-29',
  '4. Output Size': 'Full size',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2021-01-29': {'1. open': '120.22',
   '2. high': '121.3',
   '3. low': '118.9',
   '4. close': '119.11',
   '5. adjusted close': '119.11',
   '6. volume': '11825496',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2021-01-28': {'1. open': '122.8',
   '2. high': '123.2',
   '3. low': '120.07',
   '4. close': '120.08',
   '5. adjusted close': '120.08',
   '6. volume': '8084051',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2021-01-27': {'1. open': '122.08',
   '2. high': '124.79',
   '3. low': '121.6214',
   '4. close': '122.47',
   '5. adjusted close': '122.47',
   '6. volume': '12146541',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2021-01-26': {'1. open': '119.18',
   '2

In [17]:
dict_key={'1. open':'DAILYOPEN:IBM',
          '2. high':'DAILYHIGH:IBM',
          '3. low':'DAILYLOW:IBM',
          '4. close':'DAILYCLOSE:IBM',
          '5. adjusted close': 'DAILYADJCLOSE:IBM',
          '6. volume': 'DAILYVOL:IBM',
          '7. dividend amount': 'DAILYAMNT:IBM',
          '8. split coefficient': 'DAILYSPLIT:IBM',
         }

In [5]:
# Why do we want to convert to integer values? RedisTimeSeries store timestamp as Unix
## Use the epoch time to convert timestamps to integer values.
dtFmt = '%Y-%m-%d'
epoch = datetime(1970, 1, 1)

In [22]:
# Best data structure: create lists for each seperate price or dictionary?
dict_price={}

In [23]:
# Next question: How to add data such that new data is appending but old data is in front

In [24]:
for date in data['Time Series (Daily)']:
    for price_type in data['Time Series (Daily)'][date]:
        price_data = data['Time Series (Daily)'][date][price_type]
        add_price_data=( dict_key[price_type]
                        , int((datetime.strptime(str(date), dtFmt) - epoch).total_seconds()*1000)
                        , price_data)
        if price_type not in dict_price:
            dict_price[price_type]=[add_price_data]
        else:
            dict_price[price_type].append(add_price_data)

In [25]:
dict_price['2. high']

[('DAILYHIGH:IBM', 1611878400000, '121.3'),
 ('DAILYHIGH:IBM', 1611792000000, '123.2'),
 ('DAILYHIGH:IBM', 1611705600000, '124.79'),
 ('DAILYHIGH:IBM', 1611619200000, '122.59'),
 ('DAILYHIGH:IBM', 1611532800000, '119.538'),
 ('DAILYHIGH:IBM', 1611273600000, '120.7'),
 ('DAILYHIGH:IBM', 1611187200000, '132.24'),
 ('DAILYHIGH:IBM', 1611100800000, '131.06'),
 ('DAILYHIGH:IBM', 1611014400000, '129.59'),
 ('DAILYHIGH:IBM', 1610668800000, '129.24'),
 ('DAILYHIGH:IBM', 1610582400000, '130.16'),
 ('DAILYHIGH:IBM', 1610496000000, '129.75'),
 ('DAILYHIGH:IBM', 1610409600000, '129.85'),
 ('DAILYHIGH:IBM', 1610323200000, '129.675'),
 ('DAILYHIGH:IBM', 1610064000000, '129.32'),
 ('DAILYHIGH:IBM', 1609977600000, '130.46'),
 ('DAILYHIGH:IBM', 1609891200000, '131.88'),
 ('DAILYHIGH:IBM', 1609804800000, '126.68'),
 ('DAILYHIGH:IBM', 1609718400000, '125.9174'),
 ('DAILYHIGH:IBM', 1609372800000, '126.03'),
 ('DAILYHIGH:IBM', 1609286400000, '124.85'),
 ('DAILYHIGH:IBM', 1609200000000, '125.48'),
 ('DAILYH

In [20]:
list(dict_price['1. open'])

[('DAILYOPEN:IBM', 1611878400000, '120.22'),
 ('DAILYOPEN:IBM', 1611792000000, '122.8'),
 ('DAILYOPEN:IBM', 1611705600000, '122.08'),
 ('DAILYOPEN:IBM', 1611619200000, '119.18'),
 ('DAILYOPEN:IBM', 1611532800000, '118.7624'),
 ('DAILYOPEN:IBM', 1611273600000, '120.7'),
 ('DAILYOPEN:IBM', 1611187200000, '130.12'),
 ('DAILYOPEN:IBM', 1611100800000, '129.7'),
 ('DAILYOPEN:IBM', 1611014400000, '129.28'),
 ('DAILYOPEN:IBM', 1610668800000, '128.28'),
 ('DAILYOPEN:IBM', 1610582400000, '128.02'),
 ('DAILYOPEN:IBM', 1610496000000, '129.15'),
 ('DAILYOPEN:IBM', 1610409600000, '129.09'),
 ('DAILYOPEN:IBM', 1610323200000, '127.95'),
 ('DAILYOPEN:IBM', 1610064000000, '128.57'),
 ('DAILYOPEN:IBM', 1609977600000, '130.04'),
 ('DAILYOPEN:IBM', 1609891200000, '126.9'),
 ('DAILYOPEN:IBM', 1609804800000, '125.01'),
 ('DAILYOPEN:IBM', 1609718400000, '125.85'),
 ('DAILYOPEN:IBM', 1609372800000, '124.22'),
 ('DAILYOPEN:IBM', 1609286400000, '123.8'),
 ('DAILYOPEN:IBM', 1609200000000, '125.35'),
 ('DAILYOPEN:

## Rules

In [42]:
rts.create('DAILYOPEN:IBM', labels={  'SYMBOL': 'IBM'
                             , 'DESC':'OPEN'
                             , 'TIMEFRAME': '1_DAY'
                             , 'COMPANYNAME': 'IBM'})

NameError: name 'rts' is not defined